In [1]:
import pandas as pd
import numpy as np
import datetime
import mysql.connector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from joblib import load

In [3]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='951Kdroot@12',
    database='MLhack'
)

In [5]:
events_query = """
SELECT 
    c.customer_id,
    c.age,
    c.tenure,
    c.monthly_usage,
    SUM(CASE WHEN e.event_type = 'Complaints' THEN 1 ELSE 0 END) AS complaints,
    SUM(CASE WHEN e.event_type = 'Returns' THEN 1 ELSE 0 END) AS returns,
    SUM(CASE WHEN e.event_type = 'email_open' THEN 1 ELSE 0 END) AS emails_opened,
    SUM(CASE WHEN e.event_type = 'login' THEN 1 ELSE 0 END) AS daily_logins,
    SUM(CASE WHEN e.event_type = 'sensor_trigger' THEN 1 ELSE 0 END) AS sensor_triggers
FROM events e
JOIN customers c ON e.customer_id = c.customer_id
WHERE e.event_timestamp >= '2024-01-01' AND e.event_timestamp < '2024-02-01'
GROUP BY c.customer_id, c.age, c.tenure, c.monthly_usage;
"""

# Fetch data from the database using pandas
events_df = pd.read_sql(events_query, connection)

# Close the database connection
connection.close()

# Display the first few rows of the joined data
print(events_df.head())


   customer_id  age  tenure  monthly_usage  complaints  returns  \
0          240   40      40       336.3820         0.0      3.0   
1          587   63       9        43.0036         1.0      1.0   
2           84   45      50       433.1370         0.0      0.0   
3          335   45      43       288.9890         1.0      0.0   
4          225   28      17       121.2230         0.0      0.0   

   emails_opened  daily_logins  sensor_triggers  
0            0.0           0.0              1.0  
1            0.0           0.0              0.0  
2            1.0           1.0              0.0  
3            1.0           0.0              1.0  
4            0.0           1.0              0.0  


C:\Users\kdala\AppData\Local\Temp\ipykernel_26324\3076552557.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  events_df = pd.read_sql(events_query, connection)


In [7]:
imputer = SimpleImputer(strategy='mean')
events_df[['age', 'tenure', 'monthly_usage', 'complaints', 'returns', 
           'emails_opened', 'daily_logins', 'sensor_triggers']] = imputer.fit_transform(
    events_df[['age', 'tenure', 'monthly_usage', 'complaints', 'returns', 
               'emails_opened', 'daily_logins', 'sensor_triggers']]
)

In [9]:
scaler = StandardScaler()
events_df[['age', 'tenure', 'monthly_usage', 'complaints', 'returns', 
           'emails_opened', 'daily_logins', 'sensor_triggers']] = scaler.fit_transform(
    events_df[['age', 'tenure', 'monthly_usage', 'complaints', 'returns', 
               'emails_opened', 'daily_logins', 'sensor_triggers']]
)

In [11]:
import joblib

# Load the trained model from the .pkl file (or any other file you saved it as)
rf_model = joblib.load('models/random_forest_model_v1.pkl')

In [13]:
# Keep 'Customer_ID' while preparing the features for prediction
X_predict = events_df[['age', 'tenure', 'monthly_usage', 'complaints', 'returns', 
               'emails_opened', 'daily_logins', 'sensor_triggers']]




In [15]:
print("Model type:", type(rf_model)) 

Model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [17]:
# Rename columns in X_predict to match the model's expected feature names
X_predict = X_predict.rename(columns={
    'age': 'Age',
    'complaints': 'Complaints',
    'daily_logins': 'Daily_Logins',
    'emails_opened': 'Emails_Opened',
    'monthly_usage': 'Monthly_Usage',
    'sensor_triggers': 'Sensor_Triggers',
    'tenure': 'Tenure',
    'returns': 'Returns'
})

# Check the column names after renaming
print("Columns in X_predict:", X_predict.columns)



Columns in X_predict: Index(['Age', 'Tenure', 'Monthly_Usage', 'Complaints', 'Returns',
       'Emails_Opened', 'Daily_Logins', 'Sensor_Triggers'],
      dtype='object')


In [19]:
print("Columns in events_df:", events_df.columns)


Columns in events_df: Index(['customer_id', 'age', 'tenure', 'monthly_usage', 'complaints',
       'returns', 'emails_opened', 'daily_logins', 'sensor_triggers'],
      dtype='object')


In [21]:
# Now make predictions
events_df['Churn_Prediction'] = rf_model.predict(X_predict)

# Display predicted churn along with 'Customer_ID'
print(events_df[['customer_id', 'Churn_Prediction']])

     customer_id  Churn_Prediction
0            240                 0
1            587                 1
2             84                 0
3            335                 1
4            225                 0
..           ...               ...
571          307                 0
572          848                 1
573           70                 0
574          780                 0
575          248                 1

[576 rows x 2 columns]


In [23]:
# Filter at-risk customers (those with predicted churn)
at_risk_customers = events_df[events_df['Churn_Prediction'] == 1]

# Select relevant columns to send to the front end
at_risk_customers_data = at_risk_customers[['customer_id', 'Churn_Prediction']]
